In [1]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from PIL import Image
import random
from utils import (
    small_trigger_attack, watermark_trigger_attack, noised_trigger_attack,
    poison_dataset, poison_entire_testset
)

In [2]:
# config
with open("config.json", "r") as f:
    config = json.load(f)

alpha = config["alpha"]
x_coord_start = config["x_coord_start"]
y_coord_start = config["y_coord_start"]
gamma = config["gamma"]
poison_percentage = config["poison_percentage"]

In [3]:
# dataset cifar100
data_path = "./data/cifar100_data/"
train_data = datasets.CIFAR100(root=data_path, train=True, download=True)
test_data = datasets.CIFAR100(root=data_path, train=False, download=True)
train_images = [img for img, _ in train_data]
train_labels = [label for _, label in train_data]
test_images = [img for img, _ in test_data]
test_labels = [label for _, label in test_data]

100%|██████████| 169M/169M [00:22<00:00, 7.54MB/s] 


In [4]:
# trigger
trigger = Image.open("trigger.jpg")
noised_trigger = Image.open("noised_trigger_epoch_100.png")


FileNotFoundError: [Errno 2] No such file or directory: 'noised_trigger_epoch_100.png'